This notebook is used to update the field names in a feature class for the USDA Census of Agriculture. It read in the feature class and loads it into a Pandas dataframe. It uses string concatenation, regular expressions, and other string manipulation tricks to make the field names more human readable.

In [1]:
import pandas as pd
import arcpy

In [2]:
arcpy.env.overwriteOutput = True

In [3]:
in_pr_data = r"C:\Users\mark9020\Documents\ArcGIS\Projects\Ag Census\Ag Census 2022.gdb" \
    r"\ag_census_2022_county_sel_cmdty_2025R1"

In [4]:
pr_data = arcpy.da.TableToNumPyArray(in_pr_data, "*")
pr_data_df = pd.DataFrame(pr_data)
pr_data_df.head()

,OBJECTID,GEOID,Long_Desc,Value_Num,CV_PCT,Supp_Code
0,1,72002,"CATTLE, COWS - INVENTORY | TOTAL | NOT SPECIFIED",41668.0,4.3,No
1,2,72002,"CATTLE, COWS - OPERATIONS WITH INVENTORY | TOT...",281.0,4.8,No
2,3,72002,"CATTLE, COWS, BEEF - INVENTORY | TOTAL | NOT S...",2994.0,5.8,No
3,4,72002,"CATTLE, COWS, BEEF - OPERATIONS WITH INVENTORY...",95.0,8.7,No
4,5,72002,"CATTLE, COWS, BEEF, HEIFERS & HEIFER CALVES - ...",1613.0,7.3,No


In [12]:

def get_new_name(old_name: str) -> str:
    """
    Function to rename the columns in the DataFrame
    :param old_name: The old column name
    :return: The new column name
    """

    short_desc, domain_desc, cat_domain_desc = old_name.split(" | ")
    if cat_domain_desc != "NOT SPECIFIED":
        new_name = f'{short_desc}: {cat_domain_desc}'.title()
    else:
        new_name = f'{short_desc}: {domain_desc}'.title()

    corrected_name = _correct_capitalization(new_name)

    return corrected_name

def _correct_capitalization(name: str) -> str:
    """
    Function to correct the capitalization of the column names
    :param name: The column name
    :return: The corrected column name
    """
    import re

    patterns = {
        "With": "with",
        "Incl": "Incl.",
        "Excl": "Excl.",
        "Cwt": "CWT",
        "Pto": "PTO",
        " Hp,": " HP,",
        " Hp ": " HP ",
        " Ge ": " GE ",
        " Lt ": " LT ",
        " To ": " to ",
        " Gt ": " GT ",
        " Of ": " of ",
        "Lbs": "lbs.",
        " In ": " in ",
        " Bu ": " Bu. ",
        " Bu:": " Bu.:"
    }

    result = re.sub('|'.join(patterns.keys()), lambda x: patterns[x.group()], name)
    return result

In [13]:
pr_data_df['Field_Name'] = pr_data_df['Long_Desc'].apply(get_new_name)

In [14]:
pd.set_option('display.max_colwidth', None)
pr_data_df

,OBJECTID,GEOID,Long_Desc,Value_Num,CV_PCT,Supp_Code,Field_Name
0,1,72002,"CATTLE, COWS - INVENTORY | TOTAL | NOT SPECIFIED",41668.0,4.3,No,"Cattle, Cows - Inventory: Total"
1,2,72002,"CATTLE, COWS - OPERATIONS WITH INVENTORY | TOTAL | NOT SPECIFIED",281.0,4.8,No,"Cattle, Cows - Operations with Inventory: Total"
2,3,72002,"CATTLE, COWS, BEEF - INVENTORY | TOTAL | NOT SPECIFIED",2994.0,5.8,No,"Cattle, Cows, Beef - Inventory: Total"
3,4,72002,"CATTLE, COWS, BEEF - OPERATIONS WITH INVENTORY | TOTAL | NOT SPECIFIED",95.0,8.7,No,"Cattle, Cows, Beef - Operations with Inventory: Total"
4,5,72002,"CATTLE, COWS, BEEF, HEIFERS & HEIFER CALVES - INVENTORY | TOTAL | NOT SPECIFIED",1613.0,7.3,No,"Cattle, Cows, Beef, Heifers & Heifer Calves - Inventory: Total"
...,...,...,...,...,...,...,...
1368733,1368734,48279,"CATTLE, ON FEED - OPERATIONS WITH INVENTORY | TOTAL | NOT SPECIFIED",2.0,(L),No,"Cattle, On Feed - Operations with Inventory: Total"
1368734,1368735,27163,"CATTLE, ON FEED - OPERATIONS WITH SALES FOR SLAUGHTER | SALES OF CATTLE ON FEED | SALES OF CATTLE ON FEED: (100 TO 199 HEAD)",2.0,(L),No,"Cattle, On Feed - Operations with Sales For Slaughter: Sales of Cattle On Feed: (100 to 199 Head)"
1368735,1368736,33013,"CATTLE, GE 500 LBS - SALES, MEASURED IN HEAD | TOTAL | NOT SPECIFIED",831.0,(L),No,"Cattle, GE 500 lbs. - Sales, Measured in Head: Total"
1368736,1368737,18039,"CORN, SILAGE - OPERATIONS WITH AREA HARVESTED | AREA HARVESTED | AREA HARVESTED: (250 TO 499 ACRES)",5.0,52.4,No,"Corn, Silage - Operations with Area Harvested: Area Harvested: (250 to 499 Acres)"


In [15]:
# Puerto Rico data format
#
# p_data_df_trunc = pr_data_df[
#     ['Program',
#      'Region',
#      'CV____',
#      'Supp_Code',
#      'Value_Num',
#      'Long_Desc',
#      'Municipality',
#      'Region_New',
#      'GEOID',
#      'Field_Name']
#      ]

# pr_dtypes={
#     'Program': 'S255',
#     'Region': 'S255',
#     'CV____': 'S10',
#     'Supp_Code': 'S3',
#     'Value_Num': '<f8',
#     'Long_Desc': 'S512',
#     'Municipality': 'S255',
#     'Region_New': 'S255',
#     'GEOID': 'S255',
#     'Field_Name': 'S512'
# }

# USA data format
#
p_data_df_trunc = pr_data_df[
    ['GEOID',
     'Field_Name',
     'Value_Num',
     'CV_PCT',
     'Supp_Code',
     'Long_Desc'
     ]
]

pr_dtypes={
    'CV_PCT': 'S10',
    'Supp_Code': 'S3',
    'Value_Num': '<f8',
    'Long_Desc': 'S512',
    'GEOID': 'S255',
    'Field_Name': 'S512'
}


In [16]:
np_pr_data = p_data_df_trunc.to_records(index=False, column_dtypes=pr_dtypes)
np_pr_data

rec.array([(b'72002', b'Cattle, Cows - Inventory: Total', 4.1668e+04, b'4.3', b'No', b'CATTLE, COWS - INVENTORY | TOTAL | NOT SPECIFIED'),
           (b'72002', b'Cattle, Cows - Operations with Inventory: Total', 2.8100e+02, b'4.8', b'No', b'CATTLE, COWS - OPERATIONS WITH INVENTORY | TOTAL | NOT SPECIFIED'),
           (b'72002', b'Cattle, Cows, Beef - Inventory: Total', 2.9940e+03, b'5.8', b'No', b'CATTLE, COWS, BEEF - INVENTORY | TOTAL | NOT SPECIFIED'),
           ...,
           (b'33013', b'Cattle, GE 500 lbs. - Sales, Measured in Head: Total', 8.3100e+02, b'(L)', b'No', b'CATTLE, GE 500 LBS - SALES, MEASURED IN HEAD | TOTAL | NOT SPECIFIED'),
           (b'18039', b'Corn, Silage - Operations with Area Harvested: Area Harvested: (250 to 499 Acres)', 5.0000e+00, b'52.4', b'No', b'CORN, SILAGE - OPERATIONS WITH AREA HARVESTED | AREA HARVESTED | AREA HARVESTED: (250 TO 499 ACRES)'),
           (b'36039', b'Cattle, Cows, Milk - Operations with Inventory: Inventory of Milk Cows: (1 to 

In [17]:
arcpy.da.NumPyArrayToTable(np_pr_data,
                           r"C:\Users\mark9020\Documents\ArcGIS\Projects\Ag Census\Ag Census 2022.gdb" \
                            r"\ag_census_2022_county_sel_cmdty_2025R1_field_names"
                            )